<a href="https://colab.research.google.com/github/JustBoredTbh/Inventory-Qubit/blob/master/Quantitative_algorithm_for_advanced_account_rebalancing(QAABR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=10f3cef20a6e21e4aade888fe729f282a28caccc81f185702de198fca2db4add
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [2]:
import pandas as pd

def calculate_price_differential_score(df):
    # Create new columns to store the features
    df['d_ho'] = df['High'] - df['Open']
    df['d_cl'] = df['Close'] - df['Low']
    df['d_oc'] = df['Open'] - df['Close']
    df['d_hl'] = df['High'] - df['Low']
    df['d_hc'] = df['High'] - df['Close']
    df['d_ol'] = df['Open'] - df['Low']
    df['d_co'] = df['Close'] - df['Open']

    df['Price_Differential_Score'] = 0.0


    for i in range(1, len(df)):
        prev_close = df.loc[i - 1, 'Close']
        high = df.loc[i, 'High']
        low = df.loc[i, 'Low']
        open_price = df.loc[i, 'Open']
        close = df.loc[i, 'Close']
        volume = df.loc[i, 'Volume']
        atr = df.loc[i, 'ATR']

         # Calculate average volume of the past 3 candles
        if i >= 3:
            avg_volume = df.iloc[i - 3:i, df.columns.get_loc('Volume')].mean()
        else:
            avg_volume = 0.0

        df.loc[i, 'Avg_Volume'] = avg_volume

        # Calculate d_ho, d_cl, d_oc, d_hl, d_hc, d_ol, d_co
        d_ho = high - open_price
        d_cl = close - low
        d_oc = open_price - close
        d_hl = high - low
        d_hc = high - close
        d_ol = open_price - low
        d_co = close - open_price

        # Determine if it's a buy or sell candle
        is_buy_candle = close > prev_close

        # Calculate y, z, x, t
        if is_buy_candle:
            y = d_co / atr
            y = min(y, 1)  # Cap y at 1
            z = 0 if atr / d_ol < 2.5 else 1 - (d_ol / atr)
            x = 0 if atr / d_hc < 2.5 else 1 - (d_hc / atr)
        else:
            y = d_oc / atr
            y = min(y, 1)  # Cap y at 1
            z = 0 if atr / d_cl < 2.5 else 1 - (d_cl / atr)
            x = 0 if atr / d_ho < 2.5 else 1 - (d_ho / atr)

        t = 1 - (avg_volume / volume)
        t = max(t, 0)  # Ensure t is not negative

        # Calculate cc, vp, pf
        if is_buy_candle:
            cc = (d_hc / d_hl) + (d_ol / d_hl)
        else:
            cc = (d_ho / d_hl) + (d_cl / d_hl)

        vp = 1 if t == 1 and cc < 0.44 else 0

        if is_buy_candle:
            pf = (1 - (d_hc / d_hl)) + (1 - (d_ol / d_hl))
        else:
            pf = (1 - (d_ho / d_hl)) + (1 - (d_cl / d_hl))

        # Calculate price differential score
        price_differential_score = (y + z + x + t + vp + pf) / 7

        df.loc[i, 'd_ho'] = d_ho
        df.loc[i, 'd_cl'] = d_cl
        df.loc[i, 'd_oc'] = d_oc
        df.loc[i, 'd_hl'] = d_hl
        df.loc[i, 'd_hc'] = d_hc
        df.loc[i, 'd_ol'] = d_ol
        df.loc[i, 'd_co'] = d_co
        df.loc[i, 'y'] = y
        df.loc[i, 'z'] = z
        df.loc[i, 'x'] = x
        df.loc[i, 't'] = t
        df.loc[i, 'cc'] = cc
        df.loc[i, 'vp'] = vp
        df.loc[i, 'pf'] = pf
        df.loc[i, 'Price_Differential_Score'] = price_differential_score

    return df


def calculate_bollinger_band_confidence(df):
    # Create a new column to store the Bollinger Band confidence
    df['BB_Confidence'] = 0.0

    for i in range(len(df)):
        close = df.loc[i, 'Close']
        bb_high = df.loc[i, 'BB_High']
        bb_low = df.loc[i, 'BB_Low']
        price_confidence_score = df.loc[i, 'Price_Differential_Score']

        clbl = 1 if close < bb_low else 0
        clbh = 1 if close > bb_high else 0

        if clbl == 1 and price_confidence_score < 0.4:
            bb_confidence = 1
        elif clbh == 1 and price_confidence_score < 0.4:
            bb_confidence = 0
        else:
            # Calculate the distance between the closing price and the Bollinger Band high
            # Divide it by the distance between the Bollinger Band high and low
            bb_confidence = (bb_high - close) / (bb_high - bb_low)

        df.loc[i, 'BB_Confidence'] = bb_confidence

    return df


def calculate_rsi_score(df):
    # Create a new column to store the RSI score
    df['RSI_Score'] = 0.0

    for i in range(len(df)):
        rsi = df.loc[i, 'RSI']

        if rsi > 75:
            rsi_score = 0
        elif rsi < 25:
            rsi_score = 1
        else:
            rsi_score = 1 - ((rsi - 25) / 50)

        df.loc[i, 'RSI_Score'] = rsi_score

    return df


def calculate_ma_score(df):
    # Create a new column to store the MA score
    df['MA_Score'] = 0.0

    for i in range(len(df)):
        close = df.loc[i, 'Close']
        ma_values = [
            df.loc[i, 'MA_10'],
            df.loc[i, 'MA_20'],
            df.loc[i, 'MA_30'],
            df.loc[i, 'MA_40'],
            df.loc[i, 'MA_50'],
            df.loc[i, 'MA_60'],
            df.loc[i, 'MA_80'],
            df.loc[i, 'MA_100'],
            df.loc[i, 'MA_150'],
            df.loc[i, 'MA_200']
        ]

        ma_score = 0
        for j in range(len(ma_values)):
            if close > ma_values[j]:
                ma_score += 1

        # Normalize the MA score based on the number of moving averages
        ma_score /= len(ma_values)

        # Apply the stepwise scoring system
        if ma_score == 1:
            df.loc[i, 'MA_Score'] = 1
        elif ma_score > 0.9:
            df.loc[i, 'MA_Score'] = 0.98
        elif ma_score > 0.8:
            df.loc[i, 'MA_Score'] = 0.94
        elif ma_score > 0.7:
            df.loc[i, 'MA_Score'] = 0.88
        elif ma_score > 0.6:
            df.loc[i, 'MA_Score'] = 0.80
        elif ma_score > 0.5:
            df.loc[i, 'MA_Score'] = 0.69
        elif ma_score > 0.4:
            df.loc[i, 'MA_Score'] = 0.70
        elif ma_score > 0.3:
            df.loc[i, 'MA_Score'] = 0.58
        elif ma_score > 0.2:
            df.loc[i, 'MA_Score'] = 0.44
        elif ma_score > 0.1:
            df.loc[i, 'MA_Score'] = 0.28
        else:
            df.loc[i, 'MA_Score'] = 0.0

    return df


def calculate_confidence_score(df):
    # Calculate confidence score based on Bollinger Band Confidence, RSI Confidence, and Moving Average Confidence
    df['Confidence_Score'] = (df['BB_Confidence'] + df['RSI_Score'] + df['MA_Score']) / 3
    return df


In [3]:
import yfinance as yf
import pandas as pd
import ta
import numpy as np


import datetime as dt

def get_bitcoin_halvings():
    # List of known Bitcoin halving events with details
    halvings = [
        {
            'event': 'First Halving',
            'date': dt.datetime(2012, 11, 28),
            'block_number': 210000,
            'block_reward_before': 50,
            'block_reward_after': 25,
            'btc_created_per_day': 3600,
            'btc_price_before': 12.35,
            'btc_price_year_after': 964
        },
        {
            'event': 'Second Halving',
            'date': dt.datetime(2016, 7, 9),
            'block_number': 420000,
            'block_reward_before': 25,
            'block_reward_after': 12.5,
            'btc_created_per_day': 1800,
            'btc_price_before': 663,
            'btc_price_year_after': 2500
        },
        {
            'event': 'Third Halving',
            'date': dt.datetime(2020, 5, 11),
            'block_number': 630000,
            'block_reward_before': 12.5,
            'block_reward_after': 6.25,
            'btc_created_per_day': 900,
            'btc_price_before': 8500,
            'btc_price_year_after': 69000  # Peaked to $69,000 over the next several months
        }
    ]

    # Calculate the next halving date based on the previous halving
    if len(halvings) > 0:
        previous_halving = halvings[-1]
        next_halving_date = previous_halving['date'] + dt.timedelta(days=365*4)  # Add 4 years

        # Check if today's date has surpassed the estimated next halving date
        today = dt.datetime.now()
        if today >= next_halving_date:
            next_halving = {
                'event': 'Next Halving',
                'date': next_halving_date,
                'block_number': None,  # Placeholder for future data
                'block_reward_before': None,  # Placeholder for future data
                'block_reward_after': None,  # Placeholder for future data
                'btc_created_per_day': None,  # Placeholder for future data
                'btc_price_before': None,  # Placeholder for future data
                'btc_price_year_after': None  # Placeholder for future data
            }

            halvings.append(next_halving)

    return halvings, today

def fetch_cryptocurrency_data(tickers, period='max', interval='1d'):
    data_frames = {}

    for ticker in tickers:
        # Fetch data using yfinance
        data = yf.download(tickers=ticker, period=period, interval=interval)

        # Reset index to have Date as a column for easier manipulation
        data.reset_index(inplace=True)

        # Calculate technical indicators
        data['RSI'] = ta.momentum.rsi(data['Close'], window=14)
        data['ATR'] = ta.volatility.average_true_range(data['High'], data['Low'], data['Close'], window=3)
        data['BB_High'], data['BB_Low'] = ta.volatility.bollinger_hband(data['Close'], window=20), ta.volatility.bollinger_lband(data['Close'], window=20)
        data['MA_10'] = ta.trend.sma_indicator(data['Close'], window=10)
        data['MA_20'] = ta.trend.sma_indicator(data['Close'], window=20)
        data['MA_30'] = ta.trend.sma_indicator(data['Close'], window=30)
        data['MA_40'] = ta.trend.sma_indicator(data['Close'], window=40)
        data['MA_50'] = ta.trend.sma_indicator(data['Close'], window=50)
        data['MA_60'] = ta.trend.sma_indicator(data['Close'], window=60)
        data['MA_80'] = ta.trend.sma_indicator(data['Close'], window=80)
        data['MA_100'] = ta.trend.sma_indicator(data['Close'], window=100)
        data['MA_150'] = ta.trend.sma_indicator(data['Close'], window=150)
        data['MA_200'] = ta.trend.sma_indicator(data['Close'], window=200)

        # Example of calculating time-related features
        data['DayOfWeek'] = data['Date'].dt.dayofweek
        data['MonthOfYear'] = data['Date'].dt.month
        data['YearOfDecade'] = data['Date'].dt.year % 10

        # Get Bitcoin halvings and today's date
        halvings, today = get_bitcoin_halvings()

        # Find the next halving date
        next_halving_date = None
        for halving in halvings:
            if halving['event'] == 'Next Halving':
                next_halving_date = halving['date']
                break

        if next_halving_date is not None:
            # Calculate distance to next halving in days
            data['Days_to_Next_Halving'] = (next_halving_date - data['Date']).dt.days
        else:
            # If no next halving date found, fill NaN or use a default value
            data['Days_to_Next_Halving'] = pd.NA  # or use a default value like 365*4 if no future halving is found


        # Fill NaN values (if any) for technical indicators
        data.fillna(method='bfill', inplace=True)

        # Apply the feature calculations
        data = calculate_price_differential_score(data)
        data = calculate_bollinger_band_confidence(data)
        data = calculate_rsi_score(data)
        data = calculate_ma_score(data)
        data = calculate_confidence_score(data)

        # Drop NaN rows
        data.dropna(inplace=True)

        # Add ticker symbol as a column for identification
        data['Ticker'] = ticker
        # Add ticker symbol as a key for identification in the dictionary
        data_frames[ticker] = data

        print(data_frames)


    print (data_frames)
    # Concatenate all data frames into a single DataFrame
    # Find the intersection of all dates across the DataFrames
    common_dates = set.intersection(*(set(df['Date']) for df in data_frames.values())) # Use .values() on the dictionary


    # Filter each DataFrame to only include common dates
    for ticker in data_frames:
        data_frames[ticker] = data_frames[ticker][data_frames[ticker]['Date'].isin(common_dates)]

    # Create a new DataFrame to hold the combined data
    combined_data = pd.DataFrame()

    # Append rows for each date on top of each other
    for date in common_dates:
        for ticker, df in data_frames.items():
            row = df[df['Date'] == date].copy()
            row['Ticker'] = ticker
            # Replace the .append() with pd.concat()
            combined_data = pd.concat([combined_data, row], ignore_index=True)

    print(combined_data.head())

    return combined_data

# Example usage:
tickers = ['XRP-USD', 'ADA-USD', 'TON-USD', 'FTM-USD', 'AVAX-USD', 'MATIC-USD', 'UNI-USD', 'DOT-USD']
cryptocurrency_data = fetch_cryptocurrency_data(tickers)
print(cryptocurrency_data.tail())


[*********************100%%**********************]  1 of 1 completed
<ipython-input-2-4c6e6759c599>:52: RuntimeWarning: divide by zero encountered in scalar divide
  y = d_oc / atr
<ipython-input-2-4c6e6759c599>:55: RuntimeWarning: divide by zero encountered in scalar divide
  x = 0 if atr / d_ho < 2.5 else 1 - (d_ho / atr)
<ipython-input-2-4c6e6759c599>:49: RuntimeWarning: divide by zero encountered in scalar divide
  z = 0 if atr / d_ol < 2.5 else 1 - (d_ol / atr)
<ipython-input-2-4c6e6759c599>:50: RuntimeWarning: divide by zero encountered in scalar divide
  x = 0 if atr / d_hc < 2.5 else 1 - (d_hc / atr)
<ipython-input-2-4c6e6759c599>:54: RuntimeWarning: divide by zero encountered in scalar divide
  z = 0 if atr / d_cl < 2.5 else 1 - (d_cl / atr)
[*********************100%%**********************]  1 of 1 completed

{'XRP-USD':            Date      Open      High       Low     Close  Adj Close  \
1    2017-11-10  0.218256  0.219068  0.205260  0.206483   0.206483   
2    2017-11-11  0.205948  0.214456  0.205459  0.210430   0.210430   
3    2017-11-12  0.210214  0.210214  0.195389  0.197339   0.197339   
4    2017-11-13  0.197472  0.204081  0.197456  0.203442   0.203442   
5    2017-11-14  0.203679  0.213693  0.203679  0.209825   0.209825   
...         ...       ...       ...       ...       ...        ...   
2430 2024-07-05  0.433448  0.433448  0.391139  0.425492   0.425492   
2431 2024-07-06  0.425485  0.449482  0.424186  0.449053   0.449053   
2432 2024-07-07  0.449048  0.449048  0.418823  0.419823   0.419823   
2433 2024-07-08  0.419803  0.442062  0.404670  0.431480   0.431480   
2434 2024-07-09  0.431430  0.437070  0.428014  0.434302   0.434302   

          Volume        RSI       ATR   BB_High  ...         x         t  \
1      141032992  68.356951  0.000000  0.276567  ...  0.000000  1.00000


<ipython-input-2-4c6e6759c599>:52: RuntimeWarning: divide by zero encountered in scalar divide
  y = d_oc / atr
<ipython-input-2-4c6e6759c599>:54: RuntimeWarning: divide by zero encountered in scalar divide
  z = 0 if atr / d_cl < 2.5 else 1 - (d_cl / atr)
<ipython-input-2-4c6e6759c599>:50: RuntimeWarning: divide by zero encountered in scalar divide
  x = 0 if atr / d_hc < 2.5 else 1 - (d_hc / atr)
<ipython-input-2-4c6e6759c599>:49: RuntimeWarning: divide by zero encountered in scalar divide
  z = 0 if atr / d_ol < 2.5 else 1 - (d_ol / atr)
<ipython-input-2-4c6e6759c599>:55: RuntimeWarning: divide by zero encountered in scalar divide
  x = 0 if atr / d_ho < 2.5 else 1 - (d_ho / atr)
[*********************100%%**********************]  1 of 1 completed

{'XRP-USD':            Date      Open      High       Low     Close  Adj Close  \
1    2017-11-10  0.218256  0.219068  0.205260  0.206483   0.206483   
2    2017-11-11  0.205948  0.214456  0.205459  0.210430   0.210430   
3    2017-11-12  0.210214  0.210214  0.195389  0.197339   0.197339   
4    2017-11-13  0.197472  0.204081  0.197456  0.203442   0.203442   
5    2017-11-14  0.203679  0.213693  0.203679  0.209825   0.209825   
...         ...       ...       ...       ...       ...        ...   
2430 2024-07-05  0.433448  0.433448  0.391139  0.425492   0.425492   
2431 2024-07-06  0.425485  0.449482  0.424186  0.449053   0.449053   
2432 2024-07-07  0.449048  0.449048  0.418823  0.419823   0.419823   
2433 2024-07-08  0.419803  0.442062  0.404670  0.431480   0.431480   
2434 2024-07-09  0.431430  0.437070  0.428014  0.434302   0.434302   

          Volume        RSI       ATR   BB_High  ...         x         t  \
1      141032992  68.356951  0.000000  0.276567  ...  0.000000  1.00000


<ipython-input-2-4c6e6759c599>:47: RuntimeWarning: divide by zero encountered in scalar divide
  y = d_co / atr
<ipython-input-2-4c6e6759c599>:50: RuntimeWarning: invalid value encountered in scalar divide
  x = 0 if atr / d_hc < 2.5 else 1 - (d_hc / atr)
<ipython-input-2-4c6e6759c599>:49: RuntimeWarning: divide by zero encountered in scalar divide
  z = 0 if atr / d_ol < 2.5 else 1 - (d_ol / atr)
<ipython-input-2-4c6e6759c599>:55: RuntimeWarning: divide by zero encountered in scalar divide
  x = 0 if atr / d_ho < 2.5 else 1 - (d_ho / atr)
<ipython-input-2-4c6e6759c599>:50: RuntimeWarning: divide by zero encountered in scalar divide
  x = 0 if atr / d_hc < 2.5 else 1 - (d_hc / atr)
<ipython-input-2-4c6e6759c599>:57: RuntimeWarning: divide by zero encountered in scalar divide
  t = 1 - (avg_volume / volume)
<ipython-input-2-4c6e6759c599>:57: RuntimeWarning: invalid value encountered in scalar divide
  t = 1 - (avg_volume / volume)
<ipython-input-2-4c6e6759c599>:54: RuntimeWarning: divi

...

{'XRP-USD':            Date      Open      High       Low     Close  Adj Close  \
1    2017-11-10  0.218256  0.219068  0.205260  0.206483   0.206483   
2    2017-11-11  0.205948  0.214456  0.205459  0.210430   0.210430   
3    2017-11-12  0.210214  0.210214  0.195389  0.197339   0.197339   
4    2017-11-13  0.197472  0.204081  0.197456  0.203442   0.203442   
5    2017-11-14  0.203679  0.213693  0.203679  0.209825   0.209825   
...         ...       ...       ...       ...       ...        ...   
2430 2024-07-05  0.433448  0.433448  0.391139  0.425492   0.425492   
2431 2024-07-06  0.425485  0.449482  0.424186  0.449053   0.449053   
2432 2024-07-07  0.449048  0.449048  0.418823  0.419823   0.419823   
2433 2024-07-08  0.419803  0.442062  0.404670  0.431480   0.431480   
2434 2024-07-09  0.431430  0.437070  0.428014  0.434302   0.434302   

          Volume        RSI       ATR   BB_High  ...         x         t  \
1      141032992  68.356951  0.000000  0.276567  ...  0.000000  1.00000


<ipython-input-2-4c6e6759c599>:47: RuntimeWarning: divide by zero encountered in scalar divide
  y = d_co / atr
<ipython-input-2-4c6e6759c599>:50: RuntimeWarning: invalid value encountered in scalar divide
  x = 0 if atr / d_hc < 2.5 else 1 - (d_hc / atr)
<ipython-input-2-4c6e6759c599>:55: RuntimeWarning: divide by zero encountered in scalar divide
  x = 0 if atr / d_ho < 2.5 else 1 - (d_ho / atr)
<ipython-input-2-4c6e6759c599>:57: RuntimeWarning: divide by zero encountered in scalar divide
  t = 1 - (avg_volume / volume)
<ipython-input-2-4c6e6759c599>:49: RuntimeWarning: divide by zero encountered in scalar divide
  z = 0 if atr / d_ol < 2.5 else 1 - (d_ol / atr)
<ipython-input-2-4c6e6759c599>:54: RuntimeWarning: divide by zero encountered in scalar divide
  z = 0 if atr / d_cl < 2.5 else 1 - (d_cl / atr)
<ipython-input-2-4c6e6759c599>:64: RuntimeWarning: invalid value encountered in scalar divide
  cc = (d_ho / d_hl) + (d_cl / d_hl)
<ipython-input-2-4c6e6759c599>:71: RuntimeWarning:

{'XRP-USD':            Date      Open      High       Low     Close  Adj Close  \
1    2017-11-10  0.218256  0.219068  0.205260  0.206483   0.206483   
2    2017-11-11  0.205948  0.214456  0.205459  0.210430   0.210430   
3    2017-11-12  0.210214  0.210214  0.195389  0.197339   0.197339   
4    2017-11-13  0.197472  0.204081  0.197456  0.203442   0.203442   
5    2017-11-14  0.203679  0.213693  0.203679  0.209825   0.209825   
...         ...       ...       ...       ...       ...        ...   
2430 2024-07-05  0.433448  0.433448  0.391139  0.425492   0.425492   
2431 2024-07-06  0.425485  0.449482  0.424186  0.449053   0.449053   
2432 2024-07-07  0.449048  0.449048  0.418823  0.419823   0.419823   
2433 2024-07-08  0.419803  0.442062  0.404670  0.431480   0.431480   
2434 2024-07-09  0.431430  0.437070  0.428014  0.434302   0.434302   

          Volume        RSI       ATR   BB_High  ...         x         t  \
1      141032992  68.356951  0.000000  0.276567  ...  0.000000  1.00000


<ipython-input-2-4c6e6759c599>:52: RuntimeWarning: divide by zero encountered in scalar divide
  y = d_oc / atr
<ipython-input-2-4c6e6759c599>:55: RuntimeWarning: divide by zero encountered in scalar divide
  x = 0 if atr / d_ho < 2.5 else 1 - (d_ho / atr)
<ipython-input-2-4c6e6759c599>:54: RuntimeWarning: divide by zero encountered in scalar divide
  z = 0 if atr / d_cl < 2.5 else 1 - (d_cl / atr)
<ipython-input-2-4c6e6759c599>:50: RuntimeWarning: divide by zero encountered in scalar divide
  x = 0 if atr / d_hc < 2.5 else 1 - (d_hc / atr)
<ipython-input-2-4c6e6759c599>:49: RuntimeWarning: divide by zero encountered in scalar divide
  z = 0 if atr / d_ol < 2.5 else 1 - (d_ol / atr)


{'XRP-USD':            Date      Open      High       Low     Close  Adj Close  \
1    2017-11-10  0.218256  0.219068  0.205260  0.206483   0.206483   
2    2017-11-11  0.205948  0.214456  0.205459  0.210430   0.210430   
3    2017-11-12  0.210214  0.210214  0.195389  0.197339   0.197339   
4    2017-11-13  0.197472  0.204081  0.197456  0.203442   0.203442   
5    2017-11-14  0.203679  0.213693  0.203679  0.209825   0.209825   
...         ...       ...       ...       ...       ...        ...   
2430 2024-07-05  0.433448  0.433448  0.391139  0.425492   0.425492   
2431 2024-07-06  0.425485  0.449482  0.424186  0.449053   0.449053   
2432 2024-07-07  0.449048  0.449048  0.418823  0.419823   0.419823   
2433 2024-07-08  0.419803  0.442062  0.404670  0.431480   0.431480   
2434 2024-07-09  0.431430  0.437070  0.428014  0.434302   0.434302   

          Volume        RSI       ATR   BB_High  ...         x         t  \
1      141032992  68.356951  0.000000  0.276567  ...  0.000000  1.00000

In [4]:
sorted_data = cryptocurrency_data.sort_values(by='Date', ascending=True)
# Assuming 'cryptocurrency_data_sorted' is your sorted DataFrame
sorted_data.reset_index(drop=True, inplace=True)
sorted_data.head(32)

,Date,Open,High,Low,Close,Adj Close,Volume,RSI,ATR,BB_High,...,x,t,cc,vp,pf,BB_Confidence,RSI_Score,MA_Score,Confidence_Score,Ticker
0,2020-09-22,4.986754,11.463443,4.125380,5.234632,5.234632,288098840,36.137973,2.746392,5.179800,...,0.000000,1.000000,0.966220,0.0,1.033780,-0.030323,0.777241,0.88,0.542306,AVAX-USD
1,2020-09-22,4.085402,4.208596,3.918225,4.174463,4.174463,673117344,41.727625,0.464944,5.752437,...,0.926586,0.000000,0.693286,0.0,1.306714,0.875715,0.665447,0.00,0.513721,DOT-USD
2,2020-09-22,0.000718,0.000765,0.000668,0.000765,0.000765,298,53.364635,0.000251,0.001579,...,1.000000,0.329978,0.515464,0.0,1.484536,0.389156,0.432707,0.94,0.587288,UNI-USD
3,2020-09-22,0.018915,0.019723,0.018626,0.019647,0.019647,8484373,43.343478,0.002131,0.021968,...,0.964331,0.000000,0.332724,0.0,1.667276,0.572203,0.633130,0.00,0.401778,MATIC-USD
4,2020-09-22,0.037347,0.044988,0.035892,0.044705,0.044705,29314499,62.016009,0.008793,0.044727,...,0.967814,0.000000,0.191073,0.0,1.808927,0.000966,0.259680,1.00,0.420215,FTM-USD
5,2020-09-22,3.643379,3.703665,3.228452,3.415784,3.415784,364226,25.454835,0.500172,6.763277,...,0.879469,0.000000,0.521067,0.0,1.478933,0.943341,0.990903,0.00,0.644748,TON-USD
6,2020-09-22,0.080448,0.083034,0.079868,0.081754,0.081754,316907647,31.967236,0.005348,0.102722,...,0.760655,0.000000,0.587494,0.0,1.412506,1.058577,0.860655,0.00,0.639744,ADA-USD
7,2020-09-22,0.231848,0.234743,0.230237,0.233417,0.233417,1019583831,37.805614,0.009161,0.256252,...,0.855256,0.000000,0.651796,0.0,1.348204,0.934933,0.743888,0.28,0.652940,XRP-USD
8,2020-09-23,0.019633,0.019850,0.017653,0.017709,0.017709,9911559,37.619016,0.002153,0.022096,...,0.899201,0.000000,0.124260,0.0,1.875740,0.979060,0.747620,0.00,0.575560,MATIC-USD
9,2020-09-23,5.321654,5.329310,3.982604,4.118469,4.118469,173091214,36.137973,2.279830,5.179800,...,0.996642,0.445127,0.106572,0.0,1.893428,0.586927,0.777241,0.58,0.648056,AVAX-USD


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming btc_data is already loaded and processed with the necessary features including 'Date'

# Step 1: Sort data by Date (if not already sorted)
data = cryptocurrency_data.sort_values(by='Date')

# Step 2: Determine split point
# For example, you may split by a specific date or by the last N% of the data for testing
split_date = pd.to_datetime('2023-01-01')  # Example split date, adjust as needed

# Splitting data into training and testing sets
train_data = data[data['Date'] < split_date]
test_data = data[data['Date'] >= split_date]

# Alternatively, split by a percentage of data
# train_data, test_data = train_test_split(btc_data, test_size=0.2, shuffle=False)

# Step 3: Verify the split
print("Training data shape:", train_data.shape)
print("Testing data shape:", test_data.shape)

Training data shape: (5928, 46)
Testing data shape: (4432, 46)


In [12]:
import json

def execute_trade(account_ratio_allocation, current_closes, account_value_file):
  """
  Executes trades based on new allocations and calculates profit/loss.

  Args:
      account_ratio_allocation (dict): Dictionary containing new allocation ratios for each coin (e.g., {"coin1": 0.5, "coin2": 0.3}).
      current_closes (dict): Dictionary containing current closing prices for each coin (e.g., {"coin1": 10.0, "coin2": 20.0}).
      account_value_file (str): Path to the file storing the previous account value.

  Returns:
      tuple: A tuple containing (new_account_value, profit_loss, new_holdings).
  """

  # Read old account value and holdings (or initialize if not found)
  try:
    with open(account_value_file, "r") as f:
      account_value = float(f.read())
      old_value = account_value
      with open("old_holdings.json", "r") as f:
        old_holdings = json.load(f)
  except (FileNotFoundError, json.JSONDecodeError):
    account_value = 10000.0  # No previous value, assume starting fresh
    old_value = account_value
    old_holdings = {}  # No previous holdings, create empty dictionary

  # Calculate new coin amounts and total account value
  new_holdings = {}
  for coin, ratio in account_ratio_allocation.items():
    account_value += old_holdings.get(coin, 0) * current_closes[coin]  # Use old amount if available, otherwise 0
    new_amount = (ratio / 100) * account_value / current_closes[coin]
    new_holdings[coin] = new_amount

  # Calculate profit/loss based on initial account value
  profit_loss = account_value - old_value

  # Update old account value file and old holdings file
  with open(account_value_file, "w") as f:
    f.write(str(account_value))
  with open("/content/drive/MyDrive/QAABR FILES/old_holdings.json", "w") as f:
    json.dump(new_holdings, f)

  return account_value, profit_loss, new_holdings



In [13]:
# Example usage:
account_ratio_allocation = {
    'BTC': 0.6,   # 60% allocation to BTC
    'ETH': 0.4    # 40% allocation to ETH
}

current_close_data = {
    'BTC': 36000,   # Example current price of BTC
    'ETH': 2000     # Example current price of ETH
}



# Example usage (assuming data structures are populated)
new_account_value, profit_loss, new_holdings = execute_trade(account_ratio_allocation, current_close_data, "account_value.txt")


In [8]:
!pip install neat-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.4 MB/s eta 0:00:00


In [17]:
import neat
import numpy as np

combined_data = sorted_data

def eval_genomes(genomes, config, combined_data):
    for genome_id, genome in genomes:
        genome.fitness = 0.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)

        previous_account_value = 10000  # Example initial account value
        account_value_file = '/content/drive/MyDrive/QAABR FILES/account_value.txt'

        current_date = None
        account_allocation = {}

        for index, row in combined_data.iterrows():
            # If the date changes, evaluate the previous date's allocation
            if current_date != row['Date']:
                if current_date is not None:
                    # Execute trades and calculate profit/loss for the previous date
                    new_account_value, profit_loss, _ = execute_trade(account_allocation, current_close_data, account_value_file)
                    genome.fitness += profit_loss
                    previous_account_value = new_account_value

                # Reset allocation for the new date
                current_date = row['Date']
                account_allocation = {}
                current_close_data = {}

            # Get input data for NEAT
            input_data = np.array([
                row['d_ho'], row['d_cl'], row['d_oc'], row['d_hl'], row['d_hc'],
                row['d_ol'], row['d_co'], row['Price_Differential_Score'],
                row['BB_Confidence'], row['RSI_Score'], row['MA_Score'],
                row['Confidence_Score'], row['RSI'], row['ATR'], row['BB_High'],
                row['BB_Low'], row['MA_10'], row['MA_20'], row['MA_30'],
                row['MA_40'], row['MA_50'], row['MA_60'], row['MA_80'],
                row['MA_100'], row['MA_150'], row['MA_200'], row['DayOfWeek'],
                row['MonthOfYear'], row['YearOfDecade'], row['Days_to_Next_Halving']
            ])
            output = net.activate(input_data)

            # Normalize allocations to percentages
            total = sum(output)
            allocation_percentage = (output[0] / total) * 100

            # Add to current allocation and close data
            account_allocation[row['Ticker']] = allocation_percentage
            current_close_data[row['Ticker']] = row['Close']

        # Evaluate the last date's allocation
        new_account_value, profit_loss, _ = execute_trade(account_allocation, current_close_data, account_value_file)
        print(profit_loss)
        genome.fitness += profit_loss


In [ ]:
import neat
import pickle

def train_neat():
    # Load configuration from the file
    config_path = '/content/drive/MyDrive/QAABR FILES/neat_config.ini'
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)  # Load config object

    population = neat.Population(config)  # Pass config object
    population.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    population.add_reporter(stats)

    # Run NEAT algorithm
    # Pass the config and combined_data to eval_genomes
    # Pass the config and combined_data to eval_genomes using a lambda function
    winner = population.run(lambda genomes, config: eval_genomes(genomes, config, combined_data), n=50)


    # Save the best genome
    with open('best_genome.pkl', 'wb') as f:
        pickle.dump(winner, f)

    return winner

# Train the NEAT algorithm
best_genome = train_neat()


 ****** Running generation 0 ****** 

0.0
0.0
0.0
